使用numpy实现网络，手动进行前向和反向传播

In [5]:
# -*- coding: utf-8 -*-
import numpy as np

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测值y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 计算和打印损失loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 反向传播，计算w1和w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 29501433.417232275
1 24712188.86698439
2 23470088.049226277
3 22475137.270295523
4 19864223.867797244
5 15665201.025132935
6 10956759.712549748
7 7040336.248025341
8 4342054.402096892
9 2704332.2845871667
10 1760287.834819972
11 1219455.3846523303
12 899152.9560154376
13 698819.8804655324
14 564799.4059295016
15 469200.2490503923
16 397164.1290845721
17 340580.46340920014
18 294761.39597551775
19 256908.7934251557
20 225200.18467510882
21 198303.54352738598
22 175341.68739403863
23 155646.43200304604
24 138610.9395801298
25 123797.46465716175
26 110875.71119709182
27 99553.41205708351
28 89602.09032436198
29 80826.07823147147
30 73066.5272540341
31 66189.12279424196
32 60074.63574577737
33 54628.62506747614
34 49770.63091441225
35 45431.67894108072
36 41541.833222912435
37 38045.15808034959
38 34895.45849207353
39 32052.410805384883
40 29482.022159866465
41 27154.201459308657
42 25041.354614481672
43 23120.698634642285
44 21372.534496726843
45 19778.537160248547
46 18323.09842656056


使用Tensor并行计算梯度

In [6]:
# -*- coding: utf-8 -*-

import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

#创建随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 计算和打印损失
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)

    # Backprop计算w1和w2相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 38959632.0
1 39042560.0
2 43031300.0
3 41244072.0
4 30495612.0
5 16608604.0
6 7571051.5
7 3524593.75
8 1978311.25
9 1345709.75
10 1031806.75
11 838605.875
12 700586.375
13 593942.0625
14 508315.78125
15 438157.40625
16 379954.71875
17 331230.40625
18 290136.875
19 255226.46875
20 225455.203125
21 199911.4375
22 177883.3125
23 158773.15625
24 142109.359375
25 127529.3671875
26 114712.296875
27 103409.8125
28 93409.71875
29 84539.6484375
30 76665.7421875
31 69638.8125
32 63356.75390625
33 57728.0234375
34 52674.91015625
35 48125.8125
36 44024.16015625
37 40319.6796875
38 36968.3671875
39 33929.8203125
40 31172.927734375
41 28667.787109375
42 26387.810546875
43 24312.396484375
44 22419.078125
45 20689.466796875
46 19108.33203125
47 17660.15234375
48 16333.921875
49 15117.1884765625
50 13999.7890625
51 12973.310546875
52 12028.8837890625
53 11159.3046875
54 10358.4755859375
55 9620.0966796875
56 8939.0439453125
57 8310.1201171875
58 7729.32421875
59 7192.51416015625
60 6696.39697265625
6


使用pytorch自动求导

In [7]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机Tensors以保持输入和输出。
# 设置requires_grad = False表示我们不需要计算渐变
# 在向后传球期间对于这些Tensors。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 为权重创建随机Tensors。
# 设置requires_grad = True表示我们想要计算渐变
# 在向后传球期间尊重这些张贴。
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：使用tensors上的操作计算预测值y; 
      # 由于w1和w2有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 使用Tensors上的操作计算和打印丢失。
    # loss是一个形状为()的张量
    # loss.item() 得到这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播。这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()

    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 35057948.0
1 30976190.0
2 28986488.0
3 25060180.0
4 18944260.0
5 12392895.0
6 7445671.5
7 4375444.5
8 2700092.5
9 1802523.25
10 1307511.75
11 1012259.0625
12 819938.125
13 683917.0
14 581070.125
15 499924.9375
16 433855.9375
17 378948.34375
18 332686.71875
19 293381.9375
20 259727.109375
21 230748.515625
22 205656.9375
23 183860.3125
24 164806.265625
25 148116.796875
26 133476.953125
27 120545.6484375
28 109082.5078125
29 98902.3828125
30 89852.1171875
31 81764.3359375
32 74516.4765625
33 68009.453125
34 62161.46484375
35 56910.7109375
36 52168.890625
37 47878.68359375
38 43990.734375
39 40461.64453125
40 37253.9375
41 34333.703125
42 31672.720703125
43 29244.046875
44 27023.15234375
45 24991.3828125
46 23136.068359375
47 21434.474609375
48 19873.224609375
49 18438.453125
50 17118.615234375
51 15903.505859375
52 14783.9365234375
53 13751.03125
54 12797.5693359375
55 11916.44921875
56 11101.7666015625
57 10348.552734375
58 9651.798828125
59 9006.12109375
60 8407.576171875
61 7852.5224

定义自动求导函数

In [8]:
import torch

class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，
    并完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，
        并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(x)
        return x.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone()
        grad_x[x < 0] = 0
        return grad_x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 正向传播：使用张量上的操作来计算输出值y；
    # 我们通过调用 MyReLU.apply 函数来使用自定义的ReLU
    y_pred = MyReLU.apply(x.mm(w1)).mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播过程。
    loss.backward()

    with torch.no_grad():
        # 用梯度下降更新权重
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 在反向传播之后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

0 36558892.0
1 34527924.0
2 37406748.0
3 37391996.0
4 30220476.0
5 18682264.0
6 9412005.0
7 4488239.5
8 2386392.25
9 1504582.625
10 1091159.0
11 859049.4375
12 705027.6875
13 590776.1875
14 500851.46875
15 427879.9375
16 367504.96875
17 317107.0
18 274773.3125
19 238954.46875
20 208556.71875
21 182641.890625
22 160444.703125
23 141320.828125
24 124781.546875
25 110423.15625
26 97921.3984375
27 87012.1796875
28 77454.6796875
29 69065.4296875
30 61684.28125
31 55173.453125
32 49418.11328125
33 44322.8125
34 39804.6328125
35 35791.40234375
36 32220.6875
37 29037.640625
38 26197.26171875
39 23658.07421875
40 21384.869140625
41 19348.341796875
42 17521.8828125
43 15882.220703125
44 14409.9541015625
45 13086.2265625
46 11893.3125
47 10817.48828125
48 9845.9853515625
49 8967.5869140625
50 8172.99560546875
51 7453.11181640625
52 6800.77099609375
53 6209.146484375
54 5672.06591796875
55 5184.29638671875
56 4741.78125
57 4339.734375
58 3973.392578125
59 3639.79541015625
60 3335.71044921875
61 30

使用nn搭建网络

In [10]:
# -*- coding: utf-8 -*-
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

#创建输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)

     # 计算并打印损失。
     # 传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 反向传播之前清零梯度
    model.zero_grad()

    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()

    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 714.7559814453125
1 658.3048706054688
2 610.4197998046875
3 569.19140625
4 532.92529296875
5 500.64447021484375
6 471.6087646484375
7 445.2867431640625
8 421.1278076171875
9 398.64093017578125
10 377.6070251464844
11 357.8072509765625
12 339.1360778808594
13 321.5088195800781
14 304.7784729003906
15 288.7701416015625
16 273.4832458496094
17 258.989013671875
18 245.17955017089844
19 231.95794677734375
20 219.3007354736328
21 207.22193908691406
22 195.72987365722656
23 184.76060485839844
24 174.27392578125
25 164.25778198242188
26 154.6978759765625
27 145.60379028320312
28 136.95228576660156
29 128.75897216796875
30 120.97926330566406
31 113.5735092163086
32 106.55175018310547
33 99.92667388916016
34 93.66980743408203
35 87.77327728271484
36 82.21964263916016
37 76.9798583984375
38 72.06832122802734
39 67.45745086669922
40 63.118186950683594
41 59.051055908203125
42 55.246646881103516
43 51.685508728027344
44 48.35777282714844
45 45.25175476074219
46 42.353363037109375
47 39.6474761962

使用优化器

In [11]:
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包定义模型和损失函数
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用optim包定义优化器（Optimizer）。Optimizer将会为我们更新模型的权重。
# 这里我们使用Adam优化方法；optim包还包含了许多别的优化算法。
# Adam构造函数的第一个参数告诉优化器应该更新哪些张量。
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(500):

    # 前向传播：通过像模型输入x计算预测的y
    y_pred = model(x)

    # 计算并打印loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)
    optimizer.zero_grad()

    # 反向传播：根据模型的参数计算loss的梯度
    loss.backward()

    # 调用Optimizer的step函数使它所有参数更新
    optimizer.step()

0 693.4425659179688
1 675.448486328125
2 658.059326171875
3 641.2611083984375
4 625.0330810546875
5 609.3173828125
6 594.100341796875
7 579.2737426757812
8 564.8710327148438
9 551.0128784179688
10 537.6484375
11 524.712890625
12 512.0841064453125
13 499.72113037109375
14 487.66253662109375
15 476.0018615722656
16 464.6260070800781
17 453.4984130859375
18 442.6689453125
19 432.24853515625
20 422.1468505859375
21 412.37799072265625
22 402.8326110839844
23 393.5082092285156
24 384.3740539550781
25 375.4890441894531
26 366.8898010253906
27 358.5055236816406
28 350.30078125
29 342.27520751953125
30 334.4542236328125
31 326.83056640625
32 319.3756103515625
33 312.10467529296875
34 304.97894287109375
35 298.0081787109375
36 291.2157897949219
37 284.5595703125
38 278.0387268066406
39 271.6741027832031
40 265.43115234375
41 259.3054504394531
42 253.28704833984375
43 247.37493896484375
44 241.5941619873047
45 235.91749572753906
46 230.35494995117188
47 224.9030303955078
48 219.5507354736328
49 2

自定义nn

In [12]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意的（可微分的）操作。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 通过实例化上面定义的类来构建我们的模型。
model = TwoLayerNet(D_in, H, D_out)

# 构造损失函数和优化器。
# SGD构造函数中对model.parameters()的调用，
# 将包含模型的一部分，即两个nn.Linear模块的可学习参数。
loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 前向传播：通过向模型传递x计算预测值y
    y_pred = model(x)

    #计算并输出loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 768.6019287109375
1 713.5604248046875
2 666.0477905273438
3 624.3721923828125
4 587.3328857421875
5 554.0166625976562
6 523.7526245117188
7 495.9407043457031
8 469.9455261230469
9 445.4489440917969
10 422.1585693359375
11 400.1244812011719
12 379.3504638671875
13 359.6673583984375
14 340.89739990234375
15 322.9256286621094
16 305.7093811035156
17 289.2406005859375
18 273.44708251953125
19 258.3052978515625
20 243.71047973632812
21 229.74989318847656
22 216.4332275390625
23 203.71766662597656
24 191.59963989257812
25 180.07833862304688
26 169.12643432617188
27 158.70794677734375
28 148.83277893066406
29 139.47154235839844
30 130.63890075683594
31 122.30448913574219
32 114.44901275634766
33 107.05204772949219
34 100.11886596679688
35 93.60812377929688
36 87.50041961669922
37 81.77325439453125
38 76.41140747070312
39 71.4052963256836
40 66.72496032714844
41 62.36676025390625
42 58.275516510009766
43 54.44171142578125
44 50.85588455200195
45 47.515174865722656
46 44.40337371826172
47 41.

控制流和权重共享

In [13]:
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们构造了三个nn.Linear实例，它们将在前向传播时被使用。
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        对于模型的前向传播，我们随机选择0、1、2、3，
        并重用了多次计算隐藏层的middle_linear模块。
        由于每个前向传播构建一个动态计算图，
        我们可以在定义模型的前向传播时使用常规Python控制流运算符，如循环或条件语句。
        在这里，我们还看到，在定义计算图形时多次重用同一个模块是完全安全的。
        这是Lua Torch的一大改进，因为Lua Torch中每个模块只能使用一次。
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 实例化上面定义的类来构造我们的模型
model = DynamicNet(D_in, H, D_out)

# 构造我们的损失函数（loss function）和优化器（Optimizer）。
# 用平凡的随机梯度下降训练这个奇怪的模型是困难的，所以我们使用了momentum方法。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):

    # 前向传播：通过向模型传入x计算预测的y。
    y_pred = model(x)

    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 643.3082885742188
1 656.2044067382812
2 631.9678344726562
3 615.71484375
4 627.4912719726562
5 623.3151245117188
6 619.66015625
7 572.7249145507812
8 560.6119995117188
9 615.4581298828125
10 434.33258056640625
11 611.642822265625
12 382.27850341796875
13 608.0928955078125
14 606.0289916992188
15 603.5201416015625
16 491.79705810546875
17 597.3961791992188
18 581.7277221679688
19 456.6675109863281
20 583.6256103515625
21 420.06182861328125
22 396.0880432128906
23 531.31591796875
24 549.4593505859375
25 316.2558898925781
26 289.675048828125
27 261.14251708984375
28 481.8109130859375
29 455.91961669921875
30 423.6004333496094
31 390.0635681152344
32 324.8174133300781
33 338.4011535644531
34 266.1539001464844
35 233.21929931640625
36 268.837890625
37 195.0518035888672
38 213.8553466796875
39 100.35893249511719
40 128.7212677001953
41 112.99516296386719
42 211.77371215820312
43 196.0994415283203
44 125.75888061523438
45 114.25505828857422
46 87.42631530761719
47 145.68161010742188
48 64.0